In [3]:
import sys
sys.path.append("../../hw/base")

from verification_utils import CHCs, mk_int_array
import graphviz

In [4]:
from z3 import *
import z3
z3.set_param(proof=True)  # this is needed so that we can call `.proof()` later

In [5]:
stack = Array('stack', IntSort(), IntSort())
mem = Array('mem', IntSort(), IntSort())
sp = Int('sp')
r0 = Int('r0')
r1 = Int('r1')
state_vars = [stack, sp, r0, r1, mem]

a, b = input_vars = Ints('a b')

In [4]:
U = {i: Function(f"U{i}", *(v.sort() for v in [*input_vars, *state_vars]), BoolSort())
     for i in range(6)}


In [1]:
def _index_labels(instructions):
        labels = {}
        for idx, line in enumerate(instructions):
            if isinstance(line, str):
                line = line.strip()
                label = line[:-1] if line.endswith(':') else line
                labels[label] = idx + 1
        return labels

In [ ]:
def create_chc(program, input_vars, preCondition, postCondition):
    sigma = [*input_vars, *state_vars]
    U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort())
         for i in range(len(program) + 1)}
    labels = _index_labels(program)
    chc = []
    chc.append(Implies(preCondition, U[0](sigma)))
    for i in range(len(program)):
        instruction = program[i]
        match instruction[0]:
            case "PUSH":
                chc.append(Implies(
                    U[i](sigma),
                    U[i+1](*input_vars, Store(stack, sp, instruction[1]), sp + 1, r0, r1, mem)
                ))

            case "POP":
                if instruction[1] == 1:
                    v0 = Select(stack, sp - 1)
                    chc.append(Implies(
                        U[i](sigma),
                        U[i+1](*input_vars, stack, sp - 1, v0, r1, mem)
                    ))
                elif instruction[1] == 2:
                    v0 = Select(stack, sp - 2)
                    v1 = Select(stack, sp - 1)
                    chc.append(Implies(
                        U[i](sigma),
                        U[i+1](*input_vars, stack, sp - 2, v0, v1, mem)
                    ))

            case "ALU":
                match instruction[1]:
                    case "ADD":
                        v = Int(f"alu_addv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 + r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

                    case "SUB":
                        v = Int(f"alu_subv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 - r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

                    case "MUL":
                        v = Int(f"alu_mulv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 * r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

                    case "NEG":
                        v = Int(f"alu_negv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == -r0),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

                    case "AND":
                        v = Int(f"alu_andv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == If(And(r0 != 0, r1 != 0), 1, 0)),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

                    case "OR":
                        v = Int(f"alu_orv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == If(Or(r0 != 0, r1 != 0), 1, 0)),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

                    case "NOT":
                        v = Int(f"alu_notv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == If(r0 == 0, 1, 0)),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

                    case "SHL":
                        v = Int(f"alu_shlv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 << r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

                    case "SHR":
                        v = Int(f"alu_shrv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 >> r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

                    case "LT":
                        v = Int(f"alu_ltv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == If(r0 < r1, 1, 0)),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

            case "STOR":
                chc.append(Implies(
                    U[i](sigma),
                    U[i+1](*input_vars, stack, sp, r0, r1, Store(mem, r1, r0))
                ))

            case "LOAD":
                v = Int(f"mem_load_{i}")
                chc.append(Implies(
                    And(U[i](sigma), v == Select(mem, r0)),
                    U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                ))

            case "JMP":
                chc.append(Implies(
                    U[i](sigma),
                    U[labels[instruction[:-1]]](*input_vars, stack, sp, r0, r1, mem)
                ))

            case "JZ":
                chc.append(Implies(
                    And(U[i](sigma), r0 == 0),
                    U[labels[instruction[:-1]]](*input_vars, stack, sp, r0, r1, mem)
                ))

            case "JNZ":
                chc.append(Implies(
                    And(U[i](sigma), r0 != 0),
                    U[labels[instruction[:-1]]]](*input_vars, stack, sp, r0, r1, mem)
                ))

            case "RET":
                if instruction[1] == 0:
                    chc.append(Implies(
                        U[i](sigma),
                        U[r0](*input_vars, stack, sp, r0, r1, mem)
                    ))
                elif instruction[1] == 1:
                    chc.append(Implies(
                        U[i](sigma),
                        U[r0](*input_vars, Store(stack, sp, r1), sp + 1, r0, r1, mem)
                    ))

    chc.append(Implies(U[len(program)](sigma), postCondition))
    return CHCs(chc)

In [47]:
program = [ ("PUSH", 1)]
input_vars = []
preCondition = sp == 0
postCondition = And(Select(stack, 0) == 1, sp == 1)
chc = create_chc(program, input_vars, preCondition, postCondition)
chc

"sp = 0 ⇒ U0(stack, sp, r0, r1, mem)"
"U0(stack, sp, r0, r1, mem) ⇒ U1(Store(stack, sp, 1), sp + 1, r0, r1, mem)"
"U1(stack, sp, r0, r1, mem) ⇒ stack[0] = 1 ∧ sp = 1"


In [48]:
chc.solve()

U1,"(∃k!4 : ν2 = 1 ∧ ν1 = Store(k!4, 0, 1)) ∨ (∃k!4 : ν1 = Store(k!4, 0, 1) ∧ ν2 = 1)"
U0,ν1 = 0


In [7]:
program = [("PUSH", 13), ("POP",2),("ALU","MUL"), ("POP",2),("ALU","ADD"),("POP",1)]
a, b = Ints('a b')
input_vars = [a, b]
preCondition = And(stack == mk_int_array([a,b]), sp == 2)
postCondition = And(stack[sp] == a + b * 13)
chc = create_chc(program, input_vars, preCondition, postCondition)
chc

"stack = Store(Store(K(Int, 0), 0, a), 1, b) ∧ sp = 2 ⇒ U0(a, b, stack, sp, r0, r1, mem)"
"U0(a, b, stack, sp, r0, r1, mem) ⇒ U1(a, b, Store(stack, sp, 13), sp + 1, r0, r1, mem)"
"U1(a, b, stack, sp, r0, r1, mem) ⇒ U2(a, b, stack, sp - 2, stack[sp - 2], stack[sp - 1], mem)"
"U2(a, b, stack, sp, r0, r1, mem) ∧ alu_mulv_2 = r0·r1 ⇒ U3(a, b, Store(stack, sp, alu_mulv_2), sp + 1, r0, r1, mem)"
"U3(a, b, stack, sp, r0, r1, mem) ⇒ U4(a, b, stack, sp - 2, stack[sp - 2], stack[sp - 1], mem)"
"U4(a, b, stack, sp, r0, r1, mem) ∧ alu_addv_4 = r0 + r1 ⇒ U5(a, b, Store(stack, sp, alu_addv_4), sp + 1, r0, r1, mem)"
"U5(a, b, stack, sp, r0, r1, mem) ⇒ U6(a, b, stack, sp - 1, stack[sp - 1], r1, mem)"
"U6(a, b, stack, sp, r0, r1, mem) ⇒ stack[sp] = a + b·13"


In [8]:
s = chc.solve({
#    'xform.inline_eager': False,
#    'xform.inline_linear': False
})
s

U4,"ν5 = ν2[1] ∧ ν2 = Store(Store(Store(K(Int, 0), 0, ν0), 2, 13), 1, 13·ν1) ∧ ν4 = ν2[0] ∧ ν3 = 0"
U5,"ν2 = Store(Store(Store(K(Int, 0), 2, 13), 1, 13·ν1), 0, ν0 + ν5) ∧ ν5 = 13·ν1 ∧ ν3 = 1 ∧ ν4 = ν0 ∨ ν2 = Store(Store(Store(K(Int, 0), 2, 13), 1, 13·ν1), 0, ν4 + ν5) ∧ ν3 = 1 ∧ ν5 = 13·ν1 ∧ ν4 = ν0"
U3,"ν2 = Store(Store(Store(K(Int, 0), 0, ν0), 2, 13), 1, 13·ν1) ∧ ν3 = 2 ∧ ν4 = ν1 ∧ ν5 = 13 ∨ ν2 = Store(Store(Store(K(Int, 0), 0, ν0), 2, 13), 1, ν4·ν5) ∧ ν3 = 2 ∧ ν5 = 13 ∧ ν4 = ν1"
U0,"ν2 = Store(Store(K(Int, 0), 0, ν0), 1, ν1) ∧ ν3 = 2"
U2,"ν5 = ν2[2] ∧ ν2 = Store(Store(Store(K(Int, 0), 0, ν0), 1, ν1), 2, 13) ∧ ν4 = ν2[1] ∧ ν3 = 1"
U6,"ν4 = ν2[0] ∧ ν2 = Store(Store(Store(K(Int, 0), 2, 13), 1, 13·ν1), 0, ν0 + ν5) ∧ ν5 = 13·ν1 ∧ ν3 = 0 ∨ ν5 = 13·ν1 ∧ ν2 = Store(Store(Store(K(Int, 0), 2, 13), 1, 13·ν1), 0, ν0 + ν5) ∧ ν4 = ν2[0] ∧ ν3 = 0"
U1,"ν2 = Store(Store(Store(K(Int, 0), 0, ν0), 1, ν1), 2, 13) ∧ ν3 = 3 ∨ ν3 = 3 ∧ ν2 = Store(Store(Store(K(Int, 0), 0, ν0), 1, ν1), 2, 13)"


In [53]:
program = [
    ("PUSH", 0),      # 0: Push 0 to stack
    ("POP", 1),       # 1: r0 = 0
    ("JZ", 5),        # 2: if r0 == 0 → jump to 5
    ("PUSH", 999),    # 3: (should be skipped)
    ("JMP", 6),       # 4: jump to 6
    ("PUSH", 42),     # 5: target of JZ
    ("POP", 1)        # 6: r0 = 42
]

a, b = Ints("a b")  # unused input vars
input_vars = [a, b]

# Define initial state
pre = And(
    stack == mk_int_array([]),
    sp == 0
)

# Define postcondition: r0 should be 42
post = (r0 == 42)

# Generate CHCs
chc = create_chc(program, input_vars, pre, post)

# Solve
chc.solve()

U7,"ν4 = ν2[0] ∧ ν2 = Store(K(Int, 0), 0, 42) ∧ ν3 = 0 ∨ ν2 = Store(K(Int, 0), 0, 42) ∧ ν3 = 0 ∧ ν4 = 42"
U4,False
U5,"ν2 = K(Int, 0) ∧ ν3 = 0 ∧ ν4 = 0"
U3,False
U0,"ν2 = K(Int, 0) ∧ ν3 = 0"
U2,"ν2 = K(Int, 0) ∧ ν3 = 0 ∧ ν4 = 0 ∨ ν4 = ν2[0] ∧ ν2 = K(Int, 0) ∧ ν3 = 0"
U6,"ν2 = Store(K(Int, 0), 0, 42) ∧ ν3 = 1 ∧ ν4 = 0 ∨ ν3 = 1 ∧ ν4 = 0 ∧ ν2 = Store(K(Int, 0), 0, 42)"
U1,"ν2 = K(Int, 0) ∧ ν3 = 1 ∨ ν3 = 1 ∧ ν2 = K(Int, 0)"


In [55]:
program = [
    ("PUSH", 999),    # 0: value
    ("PUSH", 42),     # 1: address
    ("POP", 2),       # 2: r0 = 999, r1 = 42
    ("STOR",),        # 3: mem[42] = 999
    ("PUSH", 42),     # 4: push address again
    ("POP", 1),       # 5: r0 = 42
    ("LOAD",),        # 6: stack[sp] = mem[42]
    ("POP", 1)        # 7: r0 = mem[42] = 999
]
a, b = Ints("a b")  # unused inputs
input_vars = [a, b]

pre = And(stack == mk_int_array([]), sp == 0)
post = (r0 == 999)

chc = create_chc(program, input_vars, pre, post)
chc.solve()

U7,"(∃k!2 : ν3 = Store(Store(K(Int, 0), 1, 42), 0, ν7[42]) ∧ ν7 = Store(k!2, 42, 999) ∧ ν4 = 1 ∧ ν5 = 42 ∧ ν6 = 42) ∨ (∃k!5, k!4 : ν5 = 1 + k!4 ∧ (∃k!1 : ν9 = Store(k!1, 42, 999) ∧ k!5 = Store(Store(K(Int, 0), 1, 42), 0, 42) ∧ k!4 = 0 ∧ ν7 = 42 ∧ ν8 = 42) ∧ ν4 = Store(k!5, k!4, ν8[ν6]))"
U8,"(∃k!3 : ν3 = Store(Store(K(Int, 0), 1, 42), 0, ν7[42]) ∧ ν7 = Store(k!3, 42, 999) ∧ ν5 = ν3[0] ∧ ν4 = 0 ∧ ν6 = 42) ∨ (∃k!3, k!2 : (∃k!2 : ν5 = Store(Store(K(Int, 0), 1, 42), 0, ν9[42]) ∧ ν9 = Store(k!2, 42, 999) ∧ k!3 = 1 ∧ k!2 = 42 ∧ ν8 = 42) ∧ ν5 = -1 + k!3 ∧ ν6 = ν4[-1 + k!3])"
U4,"(∃k!1 : ν7 = Store(k!1, 42, 999) ∧ ν3 = Store(Store(K(Int, 0), 0, 999), 1, 42) ∧ ν4 = 0 ∧ ν5 = 999 ∧ ν6 = 42) ∨ (∃k!0 : ν3 = Store(Store(K(Int, 0), 0, 999), 1, 42) ∧ ν4 = 0 ∧ ν5 = 999 ∧ ν6 = 42 ∧ ν7 = Store(k!0, ν6, ν5))"
U5,"(∃k!1 : ν7 = Store(k!1, 42, 999) ∧ ν3 = Store(Store(K(Int, 0), 1, 42), 0, 42) ∧ ν4 = 1 ∧ ν5 = 999 ∧ ν6 = 42) ∨ (∃k!4, k!3 : (∃k!1 : ν9 = Store(k!1, 42, 999) ∧ k!4 = Store(Store(K(Int, 0), 0, 999), 1, 42) ∧ k!3 = 0 ∧ ν7 = 999 ∧ ν8 = 42) ∧ ν4 = Store(k!4, k!3, 42) ∧ ν5 = 1 + k!3)"
U3,"ν2 = Store(Store(K(Int, 0), 0, 999), 1, 42) ∧ ν3 = 0 ∧ ν4 = 999 ∧ ν5 = 42 ∨ ν4 = ν2[0] ∧ ν2 = Store(Store(K(Int, 0), 0, 999), 1, 42) ∧ ν5 = ν2[1] ∧ ν3 = 0"
U0,"ν2 = K(Int, 0) ∧ ν3 = 0"
U2,"ν2 = Store(Store(K(Int, 0), 0, 999), 1, 42) ∧ ν3 = 2 ∨ ν3 = 2 ∧ ν2 = Store(Store(K(Int, 0), 0, 999), 1, 42)"
U6,"(∃k!1 : ν7 = Store(k!1, 42, 999) ∧ ν3 = Store(Store(K(Int, 0), 1, 42), 0, 42) ∧ ν4 = 0 ∧ ν5 = 42 ∧ ν6 = 42) ∨ (∃k!3, k!2 : (∃k!1 : ν9 = Store(k!1, 42, 999) ∧ ν5 = Store(Store(K(Int, 0), 1, 42), 0, 42) ∧ k!3 = 1 ∧ k!2 = 999 ∧ ν8 = 42) ∧ ν5 = -1 + k!3 ∧ ν6 = ν4[-1 + k!3])"
U1,"ν2 = Store(K(Int, 0), 0, 999) ∧ ν3 = 1 ∨ ν3 = 1 ∧ ν2 = Store(K(Int, 0), 0, 999)"


In [13]:
# create_chc - test 2:
# POP
program = [("PUSH", 1), ("PUSH", 2), ("PUSH", 3),("POP",1), ("PUSH", 4), ("POP", 2)]
a, b = Int("a"), Int("b")
input_vars = [a, b]
preCondition = And(a >= 0, b >= 0)
postCondition = And(a >= 0, b >= 0)
chc = create_chc(program, input_vars, preCondition, postCondition)
chc

"a ≥ 0 ∧ b ≥ 0 ⇒ U0(a, b, stack, sp, r0, r1, mem)"
"U0(a, b, stack, sp, r0, r1, mem) ⇒ U1(a, b, Store(stack, sp, 1), sp + 1, r0, r1, mem)"
"U1(a, b, stack, sp, r0, r1, mem) ⇒ U2(a, b, Store(stack, sp, 2), sp + 1, r0, r1, mem)"
"U2(a, b, stack, sp, r0, r1, mem) ⇒ U3(a, b, Store(stack, sp, 3), sp + 1, r0, r1, mem)"
"U3(a, b, stack, sp, r0, r1, mem) ⇒ U4(a, b, stack, sp - 1, stack[sp], r1, mem)"
"U4(a, b, stack, sp, r0, r1, mem) ⇒ U5(a, b, Store(stack, sp, 4), sp + 1, r0, r1, mem)"
"U5(a, b, stack, sp, r0, r1, mem) ⇒ a ≥ 0 ∧ b ≥ 0"


In [ ]:
# create_chc - test 3:
# ALU \ ADD, SUB
program = [("PUSH", 1), ("PUSH", 2), ("ALU", "ADD"), ("ALU", "SUB"), ("PUSH", 44)]
a, b = Int("a"), Int("b")
input_vars = [a, b]
preCondition = And(a >= 0, b >= 0)
postCondition = And(a >= 0, b >= 0)
chc = create_chc(program, input_vars, preCondition, postCondition)
chc

"stack = Store(Store(K(Int, 0), 0, a), 1, b) ∧ sp = 2 ⇒ U0(a, b, stack, sp, r0, r1)"
"U0(a, b, stack, sp, r0, r1) ⇒ U1(a, b, Store(stack, sp, 1), sp + 1, r0, r1)"
"U1(a, b, stack, sp, r0, r1) ⇒ U2(a, b, Store(stack, sp, 2), sp + 1, r0, r1)"
"U2(a, b, stack, sp, r0, r1) ⇒ U3(a, b, Store(stack, sp + 1, r0 + r1), sp + 1, r0, r1)"
"U3(a, b, stack, sp, r0, r1) ⇒ U4(a, b, Store(stack, sp + 1, r0 - r1), sp + 1, r0, r1)"


In [54]:
program = [("PUSH", 13), ("POP",2),("ALU","MUL"), ("POP",2),("ALU","ADD"),("POP",1)]
a, b = Int("a"), Int("b")
input_vars = [a, b]
preCondition = And(stack == mk_int_array([a,b]), sp == 2)
postCondition = (stack[sp] == a + b * 13)
chc = create_chc(program, input_vars, preCondition, postCondition)
chc

"stack = Store(Store(K(Int, 0), 0, a), 1, b) ∧ sp = 2 ⇒ U0(a, b, stack, sp, r0, r1, mem)"
"U0(a, b, stack, sp, r0, r1, mem) ⇒ U1(a, b, Store(stack, sp, 13), sp + 1, r0, r1, mem)"
"U1(a, b, stack, sp, r0, r1, mem) ⇒ U2(a, b, stack, sp - 2, stack[sp - 2], stack[sp - 1], mem)"
"U2(a, b, stack, sp, r0, r1, mem) ⇒ U3(a, b, Store(stack, sp + 1, r0·r1), sp + 1, r0, r1, mem)"
"U3(a, b, stack, sp, r0, r1, mem) ⇒ U4(a, b, stack, sp - 2, stack[sp - 2], stack[sp - 1], mem)"
"U4(a, b, stack, sp, r0, r1, mem) ⇒ U5(a, b, Store(stack, sp + 1, r0 + r1), sp + 1, r0, r1, mem)"
"U5(a, b, stack, sp, r0, r1, mem) ⇒ stack[sp] = a + b·13"


In [57]:
s = chc.create_solver()
(res := s.check())

unsat

In [61]:
program = [("PUSH", 13), ("PUSH", 3), ("PUSH",5), ("JMP",1), ("POP",2), ("ALU","ADD"), ("POP",1)]
a, b = Int("a"), Int("b")
input_vars = [a, b]
preCondition = And(stack == mk_int_array([a,b]), sp == 2)
postCondition = (stack[sp] == a + b * 13)
chc = create_chc(program, input_vars, preCondition, postCondition)
chc

"stack = Store(Store(K(Int, 0), 0, a), 1, b) ∧ sp = 2 ⇒ U0(a, b, stack, sp, r0, r1, mem)"
"U0(a, b, stack, sp, r0, r1, mem) ⇒ U1(a, b, Store(stack, sp, 13), sp + 1, r0, r1, mem)"
"U1(a, b, stack, sp, r0, r1, mem) ⇒ U2(a, b, Store(stack, sp, 3), sp + 1, r0, r1, mem)"
"U2(a, b, stack, sp, r0, r1, mem) ⇒ U3(a, b, Store(stack, sp, 5), sp + 1, r0, r1, mem)"
"U3(a, b, stack, sp, r0, r1, mem) ⇒ U1(a, b, stack, sp, r0, r1, mem)"
"U4(a, b, stack, sp, r0, r1, mem) ⇒ U5(a, b, stack, sp - 2, stack[sp - 2], stack[sp - 1], mem)"
"U5(a, b, stack, sp, r0, r1, mem) ⇒ U6(a, b, Store(stack, sp + 1, r0 + r1), sp + 1, r0, r1, mem)"
"U6(a, b, stack, sp, r0, r1, mem) ⇒ stack[sp] = a + b·13"
